## Fetching city data

### Data for Berlin, Munich, Hamburg

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from lat_lon_parser import parse
from datetime import datetime

cities = ["Berlin", "Hamburg", "Munich"]

city_coordinates = []
city_population = []
for city in cities:
    url = f"https://en.wikipedia.org/wiki/{city}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    population_header = soup.find_all(class_='infobox-header',colspan="2")[2]
    
    city_coordinates.append ({
        'city': city,
        'latitude': parse(soup.find("span", class_="latitude").text),
        'longitude':parse(soup.find("span", class_="longitude").text),
        'country': (soup.find_all(class_="infobox-data")[0].a.text),
        'population': (population_header.find_next('td').text.replace(",", "")),
        'timestamp_population': datetime.today().date()
    })
    city_population.append ({
        'city': city,
        'population': (population_header.find_next('td').text.replace(",", "")),
        'timestamp_population': datetime.today().date()
    })

city_coordinates_df = pd.DataFrame(city_coordinates)
city_population_df = pd.DataFrame(city_population)

In [2]:
import sqlalchemy
import pymysql
import cryptography
import os 

schema = "sql_workshop"
host = "127.0.0.1"
user = "root"
password = os.getenv("con_password") 
port = 3306

engine = sqlalchemy.create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{schema}")

city_coordinates.to_sql('coordinates', if_exists='append', con=engine, index=False)
city_population.to_sql('population', if_exists='append', con=engine, index=False)

AttributeError: 'list' object has no attribute 'to_sql'

### Function for fetching city data when provided list

In [7]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from lat_lon_parser import parse
from datetime import datetime
import os
from sqlalchemy import create_engine

def retrieve_and_send_city_data():
    engine = create_connection_string()
    cities = ['New York', 'Los Angeles', 'Berlin', 'London']  # Example list of cities
    city_coordinates, city_population = fetch_city_information(cities)
    store_city_data(city_coordinates, city_population, engine)
    return "Data has been updated"

def create_connection_string():
    schema = "sql_workshop"
    host = "127.0.0.1"
    user = "root"
    password = os.getenv("con_password") 
    port = 3306
    return create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}:{port}/{schema}')

def fetch_city_information(cities):
    city_coordinates = []
    city_population = []

    for city in cities:
        url = f"https://en.wikipedia.org/wiki/{city}"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        try:
            # Latitude and longitude scraping (might not be available for all cities)
            latitude = parse(soup.find("span", class_="latitude").text) if soup.find("span", class_="latitude") else None
            longitude = parse(soup.find("span", class_="longitude").text) if soup.find("span", class_="longitude") else None
            population_header = soup.find_all(class_='infobox-header', colspan="2")[2]
            population = population_header.find_next('td').text.replace(",", "")
        except Exception as e:
            print(f"Error scraping city {city}: {e}")
            latitude, longitude, population = None, None, None

        city_coordinates.append({
            'city': city,
            'latitude': latitude,
            'longitude': longitude,
            'country': soup.find_all(class_="infobox-data")[0].a.text if soup.find_all(class_="infobox-data") else None,
            'population': population,
            'timestamp_population': datetime.today().date()
        })

        city_population.append({
            'city': city,
            'population': population,
            'timestamp_population': datetime.today().date()
        })

    return pd.DataFrame(city_coordinates), pd.DataFrame(city_population)

def store_city_data(city_coordinates, city_population, engine):
    city_coordinates.to_sql('coordinates', if_exists='append', con=engine, index=False)
    city_population.to_sql('population', if_exists='append', con=engine, index=False)



## Fetching weather forecast

In [4]:
import os
import pandas as pd
import requests
from sqlalchemy import create_engine
from pytz import timezone
from datetime import datetime


def insert_weather_data():
    engine = connection()
    cities_df = fetch_cities_data(engine)
    weather_df = fetch_weather_forecast(cities_df, engine)
    store_weather_data(weather_df, engine)
    return "Weather data successfully added"

# Establish database connection
def connection():
    schema = "sql_workshop"
    host = "127.0.0.1" 
    user = "root"
    password = os.getenv("con_password")
    port = 3306
    
    db_url = f"mysql+pymysql://{user}:{password}@{host}:{port}/{schema}"
    engine = create_engine(db_url)
    return engine

# Fetch city coordinates from database
def fetch_cities_data(engine):
    query = """
        SELECT DISTINCT c.name, co.longitude, co.latitude
        FROM city c
        JOIN coordinates co ON c.city_id = co.city_id
    """
    coordinates = pd.read_sql(query, con=engine)
    return coordinates.set_index("name")[["longitude", "latitude"]].to_dict(orient="index")

# Fetch weather data from OpenWeather API
def fetch_weather_forecast(cities_coordinates, engine):
    berlin_timezone = timezone('Europe/Berlin')
    api_key = os.getenv("weather_key") 

    weather_data = []
    retrieval_time = datetime.now(berlin_timezone).strftime('%Y-%m-%d %H:%M:%S')

    for city, coords in cities_coordinates.items():
        latitude = coords["latitude"]
        longitude = coords["longitude"]

        url = f"https://api.openweathermap.org/data/2.5/forecast?lat={latitude}&lon={longitude}&appid={api_key}&units=metric"
        response = requests.get(url)

        if response.status_code == 200:
            forecast = response.json()
            city_id_query = f"SELECT city_id FROM city WHERE name = '{city}'"
            city_id_result = pd.read_sql(city_id_query, con=engine)

            city_id = city_id_result.iloc[0]['city_id'] if not city_id_result.empty else None

            for entry in forecast["list"]:
                weather_data.append({
                    "name": city,
                    "date_time": entry["dt_txt"],
                    "min_temperature_(°C)": entry["main"]["temp_min"],
                    "max_temperature_(°C)": entry["main"]["temp_max"],
                    "feels_like_(°C)": entry['main']['feels_like'],
                    "humidity_(%)": entry["main"]["humidity"],
                    "wind_speed_(m/s)": entry["wind"]["speed"],
                    "rain_(mm_last_3h)": entry.get("rain", {}).get("3h", 0),
                    "weather": entry["weather"][0]["description"],
                    "retrieval_time": retrieval_time,
                    "city_id": city_id
                })
        else:
            print(f"Failed to fetch weather for {city}: {response.status_code}")

    return pd.DataFrame(weather_data)

# Store data in MySQL database
def store_weather_data(weather_df, engine):
    weather_df.to_sql(name='weather_forecast', con=engine, if_exists='append', index=False)


## Fetching airpot data

In [5]:
import os
import requests
import pandas as pd
from sqlalchemy import create_engine

def insert_airport_data():
    engine = connection()
    cities_df = fetch_city_coordinates(engine)
    airports_df = get_airports_with_city_id(cities_df, engine)
    store_airport_data(airports_df, engine)
    return "Airport data successfully added"

# Establish database connection
def connection():
    schema = "sql_workshop"
    host = "127.0.0.1" 
    user = "root"
    password = os.getenv("con_password")  
    port = 3306
    
    db_url = f"mysql+pymysql://{user}:{password}@{host}:{port}/{schema}"
    engine = create_engine(db_url)
    return engine

# Fetch city coordinates from database
def fetch_city_coordinates(engine):
    query = """
        SELECT DISTINCT c.name, co.longitude, co.latitude
        FROM city c
        JOIN coordinates co ON c.city_id = co.city_id
    """
    coordinates = pd.read_sql(query, con=engine)
    return coordinates.set_index("name")[["longitude", "latitude"]].to_dict(orient="index")

# Fetch airport data from an API using city coordinates
def get_airports_with_city_id(cities_coordinates, engine):
    api_key = os.getenv("flight_key")  
    
    headers = {
        "X-RapidAPI-Key": api_key,
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
    }
    
    url = "https://aerodatabox.p.rapidapi.com/airports/search/location"
    
    all_airports = []

    for city_name, coords in cities_coordinates.items():
        latitude = coords["latitude"]
        longitude = coords["longitude"]
        
        querystring = {"lat": latitude, "lon": longitude, "radiusKm": "50", "limit": "3", "withFlightInfoOnly": "true"}

        response = requests.get(url, headers=headers, params=querystring)

        if response.status_code == 200:
            data = response.json()
            airports = pd.json_normalize(data.get('items', []))
            
            # Fetch city_id from the database
            city_id_query = f"SELECT city_id FROM city WHERE name = '{city_name}'"
            city_id_result = pd.read_sql(city_id_query, con=engine)

            if not city_id_result.empty:
                city_id = city_id_result.iloc[0]['city_id']
                airports['city_id'] = city_id  # Add city_id to the dataframe
            else:
                print(f"City {city_name} not found in the city table.")
            
            all_airports.append(airports)
        else:
            print(f"Failed to fetch data for city: {city_name}, status code: {response.status_code}")
    
    if all_airports:
        return pd.concat(all_airports, ignore_index=True)
    else:
        return pd.DataFrame()  # Return an empty DataFrame if no data is fetched

# Store airport data in the database
def store_airport_data(airports_df, engine):
    airports_df.to_sql(name='airport_data', con=engine, if_exists='append', index=False)


## Fetching flight arrival data

In [6]:
import os
import pandas as pd
import requests
from sqlalchemy import create_engine
from pytz import timezone
from datetime import datetime, timedelta


def insert_flight_data():
    engine = connection()
    airport_dict = fetch_airport_data(engine)
    flight_df = fetch_flight_arrivals(airport_dict, engine)
    store_flight_data(flight_df, engine)
    return "Flight data successfully added"

# Establish database connection
def connection():
    schema = "sql_workshop"
    host = "127.0.0.1" 
    user = "root"
    password = os.getenv("con_password") 
    port = 3306
    
    db_url = f"mysql+pymysql://{user}:{password}@{host}:{port}/{schema}"
    engine = create_engine(db_url)
    return engine

def fetch_airport_data(engine):
    query = """SELECT DISTINCT city_id, iata FROM airport """
    airports = pd.read_sql(query, con=engine)
    return airports.to_dict(orient='index')

# Fetch flight arrival data from Aerodatabox API
def fetch_flight_arrivals(airport_dict, engine):
    berlin_timezone = timezone('Europe/Berlin')
    today = datetime.now(berlin_timezone).date()
    tomorrow = today + timedelta(days=1)

    times = [["00:00", "11:59"],
             ["12:00", "23:59"]]

    arrivals = {}
    arrival_data = []
    api_key = os.getenv("flight_key") 

    for city, airport in airport_dict.items():
        iata = airport['iata']
        city_id = airport['city_id']

        for time_range in times:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/iata/{iata}/{tomorrow}T{time_range[0]}/{tomorrow}T{time_range[1]}"

            querystring = {
                "direction": "Arrival",
                "withCancelled": "true",
                "withCodeshared": "true",
                "withCargo": "false",
                "withPrivate": "true",
                "withLocation": "false"
            }

            headers = {
                "x-rapidapi-key": api_key,
                "x-rapidapi-host": "aerodatabox.p.rapidapi.com"
            }

            response = requests.get(url, headers=headers, params=querystring)
            retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")

            if response.status_code == 200:
                data = response.json()
                if 'arrivals' in data:
                    if city_id not in arrivals:
                        arrivals[city_id] = {'iata': iata, 'arrivals': []}
                    arrivals[city_id]['arrivals'].extend(data['arrivals'])

            else:
                print(f"Failed to fetch arrivals for {city}: {response.status_code}")

    for city_id, arrival in arrivals.items():
        for entry in arrival["arrivals"]:
            movement = entry.get('movement', {})
            arrival_data.append({
                'city_id': city_id,
                'icao': movement.get('airport', {}).get('icao', 'Unknown'),
                'iata': movement.get('airport', {}).get('iata', 'Unknown'),
                'name': movement.get('airport', {}).get('name', 'Unknown'),
                'timeZone': movement.get('airport', {}).get('timeZone', 'Unknown'),
                'scheduledTime_local': movement.get('scheduledTime', {}).get('local', 'Unknown'),
                'revisedTime_local': movement.get('revisedTime', {}).get('local', 'Unknown'),
                'terminal': movement.get('terminal', 'Unknown'),
                "data_retrieved_at": retrieval_time
            })

    df_arrivals = pd.DataFrame(arrival_data)

    df_arrivals['scheduledTime_local'] = pd.to_datetime(df_arrivals['scheduledTime_local'], errors='coerce').dt.tz_localize(None)
    df_arrivals['revisedTime_local'] = pd.to_datetime(df_arrivals['revisedTime_local'], errors='coerce').dt.tz_localize(None)
    df_arrivals["data_retrieved_at"] = pd.to_datetime(df_arrivals["data_retrieved_at"])

    return df_arrivals

# Store data in MySQL database
def store_flight_data(flight_df, engine):
    flight_df.to_sql(name='flight', con=engine, if_exists='append', index=False)
